In [ ]:
"""
Purpose: To Run the neuron preprocessing


"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

# configuring the virtual module

In [2]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 30)
print(f"Sleeping {random_sleep_sec} sec before conneting")
#time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-21 23:03:59,039 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-21 23:03:59,040 - settings - Setting database.user to celiib
INFO - 2020-11-21 23:03:59,041 - settings - Setting database.password to newceliipass
INFO - 2020-11-21 23:03:59,046 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-21 23:03:59,047 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-21 23:03:59,059 - connection - Connect

Sleeping 0 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons
Connecting celiib@at-database.ad.bcm.edu:3306


# Defining Our Table

In [3]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [4]:
from soma_extraction_utils import *
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 1610


In [5]:
#so that it will have the adapter defined
from datajoint_utils import *

In [6]:
#minnie.Decomposition.drop()
#schema.external['decomposition'].delete(delete_external_files=True)

In [7]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ---
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branche

    
    """

    key_source =  ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0").proj())
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n----- Working on {segment_id}-------")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

        #3) Get the somas info *************************** Need to change this when actually run *******************
        somas = du.get_soma_mesh_list(segment_id) 
        print(f"somas = {somas}")
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"]

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")



        #7) Pass stats and file location to insert
        new_key = dict(key,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Cleaning the external files

In [13]:
(minnie.schema.jobs & "table_name='__decomposition'").delete()
#((schema.jobs & "table_name = '__decomposition'") & "timestamp>'2020-11-16 00:26:00'").delete()

In [12]:
import preprocessing_vp2 as pre
import compartment_utils as cu
cu = reload(cu)
pre = reload(pre)

In [14]:
import time
import random

start_time = time.time()
#time.sleep(random.randint(0, 900))
print('Populate Started')
Decomposition.populate(reserve_jobs=True, suppress_errors=False, order='random')
print('Populate Done')

print(f"Total time for Decomposition populate = {time.time() - start_time}")

Populate Started


----- Working on 864691135638993211-------
somas = [[<trimesh.Trimesh(vertices.shape=(10465, 3), faces.shape=(20678, 3))>], array([20.8582]), array([0.721])]

--- Beginning preprocessing of 864691135638993211---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(10465, 3), faces.shape=(20678, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(10465, 3), faces.shape=(20678, 3))>]
soma_mesh_list_centers = [array([651117.30981366, 974868.62249402, 600757.07537506])]
xvfb-run -n 1610 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Neuron_Processing/temp/neuron_10993.off -o /notebooks/Platinum_Datajoint/Neuron_Processing/temp/neuron_10993_fill_holes.off -s /notebooks/Platinum_Datajoint/Neuron_Processing/temp/fill_holes_974741.mls

---- meshlab output -----
QS

  0%|          | 0/1 [00:00<?, ?it/s]

Total time for mesh_pieces_connectivity= 0.5221388339996338
# of insignificant_limbs = 3 with trimesh : [<trimesh.Trimesh(vertices.shape=(775, 3), faces.shape=(1405, 3))>, <trimesh.Trimesh(vertices.shape=(743, 3), faces.shape=(1310, 3))>, <trimesh.Trimesh(vertices.shape=(341, 3), faces.shape=(554, 3))>]
# of not_processed_soma_containing_meshes = 0 with trimesh : []


----- Working on Proper Limb # 0 ---------
meshparty_segment_size = 100
Time for preparing soma vertices and root: 1.0251998901367188e-05
cc_vertex_thresh = 10


100%|██████████| 1/1 [00:00<00:00,  9.99it/s]

Time for 1st pass MP skeletonization: 0.2300276756286621
connecting at the root
branches_touching_root = [4]
length of Graph = 2507
max(kept_branches_idx) = 4, len(kept_branches_idx) = 5
empty_indices % = 0.0
 conflict_indices % = 0.005113135468704085


...

100%|██████████| 1/1 [00:05<00:00,  5.70s/it]

Time for 1st pass MP skeletonization: 7.668431520462036
connecting at the root
branches_touching_root = [61]
length of Graph = 21818
Working on path [13331. 13348. 13363. 13377. 13396. 13413. 13433. 13442.]
path_degrees = [3, 2, 2, 2, 2, 2, 2, 3]
max(kept_branches_idx) = 61, len(kept_branches_idx) = 61
empty_indices % = 0.0
 conflict_indices % = 0.007639157026168539



AFTER face_lookup_resolved_test



Decomposing first pass: 14.388145685195923
Attempting to use MeshAfterParty Skeletonization and Mesh Correspondence
Another print
Found len(mesh_large_idx) MAP candidates: [25264, 2144, 4595, 880, 32881, 3528, 3067, 2183, 902, 588, 26918, 2771, 10119, 12518, 2465, 49861, 36749, 15677, 2743, 10038, 3824, 6811, 12473, 15498, 7621, 27282, 1053, 21119, 4694]
mesh_large_connectivity: 2.041611433029175
Finding MAP candidates connected components: 0.00036978721618652344
len(filtered_pieces) = 1
skeleton_connectivity_MP : 0.3985581398010254
Grouping MP Sublimbs by Graph: 0.07995986938476562
Divinding into MP and MAP pieces: 9.5367431640625e-07
--- Working on MAP piece 0---
MAP Filtering Soma Pieces: 0.023604393005371094
inside skeletonize_connected_branch and use_surface_after_CGAL=False, surface_reconstruction_size=1000
     Starting Screened Poisson
xvfb-run -n 1610 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Neuron_Processing/temp/None.off -o /notebooks/Pla

Exception: Something went wrong in remove_cycles_from_skeleton

# Clearing Files

In [10]:
curr_path = Path("/mnt/dj-stor01/platinum/minnie65/02/decomposition/")
for f in curr_path.iterdir():
    print(f'{f}')
    #f.unlink()
    

# Checking the Table

In [ ]:
minnie.Decomposition()

In [ ]:
du = reload(du)
from datajoint_utils import *
my_dj_neuron = (minnie.Decomposition & "segment_id=864691134221889045").fetch1("decomposition")

import neuron_visualizations as nviz
nviz.visualize_neuron(my_dj_neuron,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all")

# Debugging the neuron creation

In [ ]:
#1) Get the segment id from the key
segment_id = 864691137196899649
description = "0_25"
print(f"\n\n----- Working on {segment_id}-------")
global_start = time.time()

#2) Get the decimated mesh
current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

#3) Get the somas info *************************** Need to change this when actually run *******************
somas = du.get_soma_mesh_list(segment_id) 
print(f"somas = {somas}")
#4) Run the preprocessing


total_neuron_process_time = time.time()

In [ ]:
import preprocessing_vp2 as pre
import soma_extraction_utils as sm

import neuron
tu = reload(tu)
du = reload(du)
sm = reload(sm)
pre = reload(pre)

neuron = reload(neuron)
print(f"\n--- Beginning preprocessing of {segment_id}---")
recovered_neuron = neuron.Neuron(
mesh = current_neuron_mesh,
somas = somas,
segment_id=segment_id,
description=description,
suppress_preprocessing_print=False,
suppress_output=False,
calculate_spines=True,
widths_to_calculate=["no_spine_median_mesh_center"]

        )

In [ ]:
import neuron_visualizations as nviz

In [ ]:
nviz.visualize_neuron(recovered_neuron,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all")